In [ ]:
! pip install xgboost
! pip install pandas
! pip install scikit-learn
! pip install pyarrow

In [1]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, mean, stddev
from pyspark.sql import Window
from pyspark.sql.types import DoubleType, FloatType, LongType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from xgboost.spark import SparkXGBRegressor

import pyspark.sql.functions as F
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import Bucketizer
from pyspark.sql import DataFrame
from typing import Iterable
import pandas as pd

from xgboost.spark import SparkXGBRegressor
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName("check_files").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/17 23:06:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_location1='/cephfs/summary_stats/train/summary_stats_parquet_164_25.parquet'

In [4]:
df = spark.read.format('parquet').load(data_location1)
df.show(3)

+-----+-----+------------------+------------------+------+----+-----+--------------------+--------------------+-----------------+------------------+----------+----------+---------------+------+
|NumId|Chunk|              Mean|            StdDev|Median| Min|  Max|        AvgFirstDiff|          AvgSecDiff|     StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|
+-----+-----+------------------+------------------+------+----+-----+--------------------+--------------------+-----------------+------------------+----------+----------+---------------+------+
|   19|    1|138.95138888888889| 30.28237142249909| 133.0|68.0|198.0|-0.08333333333333333|0.034722222222222224|4.810702354423639| 4.235922690223849|         0|         1|              1|    23|
|   19|    2|140.04513888888889|41.219663671012334| 131.0|83.0|247.0| 0.17708333333333334|0.017361111111111112| 3.96850350067166|3.6725335151182836|         0|         0|              0|     1|
|   19|    3|139.7256944444444

In [5]:
class ColumnScaler(Transformer):
    def _transform(self, df):
        double_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
        float_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, FloatType)]
        long_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, LongType)]

        all_numerical=list(set(double_cols+float_cols+long_cols))
        all_numerical.remove('target')
        
        for num_column in all_numerical:
            input_col = f"{num_column}"
            output_col = f"scaled_{num_column}"

            w = Window.partitionBy('NumId')

            mu = mean(input_col).over(w)
            sigma = stddev(input_col).over(w)

            df=df.withColumn(output_col, (col(input_col) - mu)/(sigma))
            
        return df

In [6]:
double_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
float_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, FloatType)]
long_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, LongType)]

all_numerical=list(set(double_cols+float_cols+long_cols))
all_numerical.remove('target')

featureArr = [('scaled_' + f) for f in all_numerical]

columns_scaler=ColumnScaler()

va2 = VectorAssembler(inputCols=featureArr, outputCol="features")

In [7]:
stages=[columns_scaler]+[va2]

In [ ]:
pipeline=Pipeline(stages=stages)

model_1=pipeline.fit(df)
model_df=model_1.transform(df)

In [ ]:
model_df.show(2)

In [ ]:
model_df.select('features').show(2, truncate=False)

In [8]:
features_col="features"
label_name="target"

# xgb_regression=SparkXGBRegressor(features_col=features_col, 
#                                   label_col=label_name,
#                                   num_workers=2,
#                                   random_state=123,
#                                   use_gpu=True)
xgb_regression=SparkXGBRegressor(features_col=features_col, 
                                  label_col=label_name,
                                  random_state=123,
                                  use_gpu=False)



stages.append(xgb_regression)


In [9]:
stages

[ColumnScaler_6518e0478184,
 VectorAssembler_015139367cd0,
 SparkXGBRegressor_cea6cbd16698]

In [10]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(df)

23/05/17 23:07:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[23:07:28] task 0 got new rank 0                                    (0 + 1) / 1]


[23:07:29] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


In [ ]:
model_stages

In [ ]:
pipeline=Pipeline(stages=model_stages)

model=pipeline.fit(model_df)

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(df)

In [ ]:
model.transform(df)

In [ ]:
df.schema

In [ ]:
double_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
float_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, FloatType)]
long_cols=[f.name for f in df.schema.fields if isinstance(f.dataType, LongType)]

all_numerical=list(set(double_cols+float_cols+long_cols))
all_numerical.remove('target')

featureArr = [('scaled_' + f) for f in all_numerical]
        

In [ ]:
all_numerical

In [ ]:
for num_column in all_numerical:
    input_col = f"{num_column}"
    output_col = f"scaled_{num_column}"

    w = Window.partitionBy('NumId')

    mu = mean(input_col).over(w)
    sigma = stddev(input_col).over(w)

    df=df.withColumn(output_col, (col(input_col) - mu)/(sigma))
    

In [ ]:
va2 = VectorAssembler(inputCols=featureArr, outputCol="features")

stages = [va2]



In [ ]:
features_col="features"
label_name="target"

xgb_classifier=SparkXGBRegressor(features_col=features_col, 
                                  label_col=label_name,
                                  num_workers=4,
                                  random_state=123,
                                  use_gpu=True)



stages.append(xgb_classifier)

In [ ]:
stages

In [ ]:
pipeline=Pipeline(stages=stages)

In [ ]:
model=pipeline.fit(ml_df)

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(df)

model.transform(df).select(featureArr).show(3, truncate=False)